In [1]:
import pandas as pd
import os
import requests
from PIL import Image
from io import BytesIO

In [2]:
from google.colab import files
uploaded = files.upload()

Saving offres_tunisair.csv to offres_tunisair.csv


In [3]:
# Charger le fichier CSV
df = pd.read_csv('offres_tunisair.csv', sep=';', encoding='ISO-8859-1')


In [4]:
# Fonction pour déterminer la classe
def get_price_class(price):
    if price < 500000:
        return "Classe_0"
    elif price <= 800000:
        return "Classe_1"
    else:
        return "Classe_2"

In [5]:
def clean_price(p):
    try:
        p = p.replace(" TND", "").replace(",", ".").strip()
        return float(p)
    except:
        return None  # ou np.nan si tu veux filtrer avec .dropna()


In [6]:
# Créer les dossiers si non existants
os.makedirs("images/Classe_0", exist_ok=True)
os.makedirs("images/Classe_1", exist_ok=True)
os.makedirs("images/Classe_2", exist_ok=True)

In [7]:
# Appliquer nettoyage
df['Prix_nettoye'] = df['Prix'].apply(clean_price)


In [8]:
# Boucle de téléchargement d’images
for index, row in df.iterrows():
    try:
        prix = clean_price(row['Prix'])
        if prix is None:
            raise ValueError("Prix invalide")
        classe = get_price_class(prix)
        url = row['Image']
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        img = img.resize((224, 224))
        img.save(f"images/{classe}/{index}.jpg")
    except Exception as e:
        print(f"Erreur pour l'image {index} : {e}")

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [10]:
train_generator = train_datagen.flow_from_directory(
    'images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 8 images belonging to 3 classes.


In [11]:
val_generator = train_datagen.flow_from_directory(
    'images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2 images belonging to 3 classes.


In [12]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [13]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
model = Model(inputs=base_model.input, outputs=predictions)

# Geler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

In [15]:
# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [16]:
# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])



In [17]:
# Résumé du modèle
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

input_layer : C'est la couche d'entrée du réseau, qui prend une image de taille (224, 224, 3).

Conv1 (Conv2D) : Applique une convolution avec 32 filtres, réduisant la taille de l'image à (112, 112, 32).

bn_Conv1 (BatchNormalization) : Effectue une normalisation par lots, ce qui permet de stabiliser l'apprentissage.

Conv1_relu (ReLU) : Applique une activation ReLU pour introduire de la non-linéarité.

expanded_conv_depthwise (DepthwiseConv2D) : Applique une convolution séparée par canal, réduisant la taille à (112, 112, 32).

expanded_conv_depthwise_bn (BatchNormalization) : Normalisation par lots sur cette couche.

expanded_conv_depthwise_relu (ReLU) : Activation ReLU sur la sortie de la convolution séparée.

expanded_conv_proj (Conv2D) : Applique une convolution pour projeter les caractéristiques à une dimension réduite, ici à (112, 112, 16).

expanded_conv_proj_bn (BatchNormalization) : Normalisation par lots après la projection.

block_1_expand (Conv2D) : Applique une convolution avec 96 filtres pour étendre les dimensions des caractéristiques à (112, 112, 96).

block_1_expand_BN (BatchNormalization) : Normalisation par lots après l'extension.

block_1_expand_relu (ReLU) : Activation ReLU après l'extension.

block_1_pad (ZeroPadding2D) : Ajout de padding pour ajuster les dimensions de l'image avant la convolution suivante.

In [20]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [19]:
# Créer une copie nettoyée du DataFrame original
df_clean = df[['Prix', 'Prix_nettoye']].copy()
df_clean['Classe_reelle'] = df_clean['Prix_nettoye'].apply(get_price_class)
df_clean['Index'] = df_clean.index.astype(str)
df_clean['Chemin_image'] = df_clean.apply(lambda row: f"images/{row['Classe_reelle']}/{row.name}.jpg", axis=1)

In [21]:
# Fonction de prédiction sur une seule image
def predict_class(image_path):
    try:
        img = load_img(image_path, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0) / 255.0
        preds = model.predict(x, verbose=0)
        classe_predite = np.argmax(preds, axis=1)[0]
        return f"Classe_{classe_predite}"
    except Exception as e:
        print(f"Erreur avec {image_path} : {e}")
        return "Erreur"

In [22]:

# Prédire les classes
df_clean['Classe_predite'] = df_clean['Chemin_image'].apply(predict_class)

# Nettoyer les lignes où la prédiction a échoué
df_resultats = df_clean[df_clean['Classe_predite'] != "Erreur"].copy()

# Réorganiser les colonnes
df_resultats_final = df_resultats[['Index', 'Prix', 'Prix_nettoye', 'Classe_reelle', 'Classe_predite', 'Chemin_image']]

# Exporter dans un fichier CSV lisible
df_resultats_final.to_csv('resultats_predictions_tunisair_nettoye.csv', index=False, encoding='utf-8-sig', sep=';')

print("✅ Fichier CSV propre enregistré : resultats_predictions_tunisair_nettoye.csv")

✅ Fichier CSV propre enregistré : resultats_predictions_tunisair_nettoye.csv
